# Lista 3

### Zaimplementowane biblioteki

In [1]:
import os
from zipfile import ZipFile
import datetime
import time
from PyPDF2 import PdfFileMerger
import cv2 as cv
from PIL import Image
import qrcode
import webbrowser
from bs4 import BeautifulSoup
import requests

### Zadanie 1

In [ ]:
def zip(sciezka, rozszerzenie):
    str_sciezka = str(sciezka)           
    data = str(datetime.date.today())   #zapisujemy dzisiejsza date
    os.mkdir('c:\\copy-' + data)   #tworzymy folder z podana nazwa
    lista = []
    nazwa = data + '_' + os.path.basename(sciezka) + '.zip'  #tworzymy nazwe zipu  
    for root, sciezka, pliki in os.walk(sciezka):        #petla w ktorej zapisujemy kazdy plik z podanym rozszerzeniem i czasem od ostatniej modyfikacji krotszym niz 3 dni do listy
        for plik in pliki:
            nazwa_pliku, rozszerzenie_pliku = os.path.splitext(plik) #zmienna zapisujaca rozszerzenie kazdego pliku z podanej sciezki
            os.chdir(str_sciezka)
            czas_zmiany = os.path.getmtime(plik)   #zmienne zapisujace czas modyfikacji  i czas lokalny
            czas_teraz = time.mktime(time.localtime())
            roznica = round(czas_teraz - czas_zmiany) 
            if rozszerzenie_pliku == rozszerzenie and roznica < 259200: #warunek ograniczajacy zapisywanie plikow w zipie
                sciezka_pliku = os.path.join(root, plik)
                lista.append(sciezka_pliku)
    os.chdir('c:\\copy-' + data) #zmiana sciezki aby zip zrobil sie w odpowiednim folderze
    with ZipFile(nazwa, 'w') as zip:  #zapisujemy zip
        for element in lista:
            zip.write(element)
zip('sciezka', 'rozszerzenie')

### Zadanie 2

In [ ]:
def win_to_unix(sciezka):
    koniec_win = b'\r\n'  #zapisujemy koncowki linii windowsa i unixa w systemie bitowym
    koniec_unix = b'\n'
    with open(sciezka, 'rb') as plik: #otwieramy podany plik
        tekst = plik.read()

    zmiana = tekst.replace(koniec_win, koniec_unix) #zamieniamy koncowki linii w pliku i go zapisujemy
    with open(sciezka, 'wb') as plik:
        plik.write(zmiana)

win_to_unix( "sciezka" )

def unix_to_win(sciezka):  #druga funkcja dziala analogicznie do pierwszej
    koniec_win = b'\r\n'
    koniec_unix = b'\n'
    with open(sciezka, 'rb') as plik:
        tekst = plik.read()

    zmiana = tekst.replace(koniec_unix, koniec_win)
    with open(sciezka, 'wb') as plik:
        plik.write(zmiana)

unix_to_win( "sciezka" )

### Zadanie 3

In [ ]:
def pdf(pliki: list, zapis):
    
    for plik in pliki:
        PdfFileMerger().append(plik) #w pętli dodajemy do siebie kazdy podany plik i zapisujemy w jednym
    PdfFileMerger().write(zapis)
    PdfFileMerger().close()

pdf(['sciezka1.pdf', 'sciezka2.pdf'], "zapis.pdf")

### Zadanie 4

In [ ]:
def generator(strona):
    qr = qrcode.QRCode()
    qr.add_data(strona)    #tworzymy kod qr z podanej strony
    qr.make(fit = True)
    kod = qr.make_image()
    kod.save("qrcode.jpg")  #zapisujemy kod na dysku
    obraz = Image.open('sciezka do kodu qr') #pokazujemy wytworzony kod qr
    obraz.show()

generator('strona')

def czytnik(image):
    qr = cv.imread(image)   #odczytujemy kod qr
    czytnik_qr = cv.QRCodeDetector()    
    strona, tablica, kod = czytnik_qr.detectAndDecode(qr)   #z funkcji zwracajacej trzy zmienne zapisujemy link strony i go wyswietlamy
    print("Zakodowana strona to: " + strona)

czytnik('sciezka do kodu qr')

### Zadanie 5

In [ ]:
def nawiasy():
    otwierajace = ['[', '{', '(', '<']  #tworzymy listy mozliwych nawiasow otwierajacych i zamykajacych
    zamykajace = [']', '}', ')', '>']
    lista = []
    wyrazenie = input("Podaj wyrazenie: ")    #podajemy nasze wyrazenie jako input z klawiatury uzytkownika
    for element in wyrazenie:
        if element in otwierajace:    #w petli dla kazdego elementu wyrazenia zapisujemy je w zaleznosci od tego czy jest to nawias otwierajacy czy zamykajacy
            lista.append(element)       #jesli jest to nawias otwierajacy to dodajemy go do listy
        elif element in zamykajace:
            if len(lista) > 0 and otwierajace[zamykajace.index(element)] == lista[-1]:
                lista.pop()  #jesli jest to nawias zamykajacy sprawdzamy czy w liscie znajduja sie inne wyrazenia i porownujemy nasz nawias z ostatnim elementem z listy
            else:
                return 'Wyrazenie nie jest poprawne' #jesli nawias otwierajacy i zamykajacy ktore znajduja sie obok siebie nie sa takie same to wyrazenie nie jest poprawne
    if len(lista) == 0: #jesli udalo nam sie wyrzucic kazdy nawias otwierajacy i nie ma wiecej elementow w wyrazeniu to jest ono poprawne
        return 'Wyrazenie jest poprawne'
    else:
        return 'Wyrazenie nie jest poprawne' #jesli zostal jakis nawias otwierajacy to wyrazenie nie jest poprawne

nawiasy()

### Zadanie 6

In [2]:
def wikipedia():
    strona = requests.get('https://en.wikipedia.org/wiki/Special:Random') #losujemy strone
    zawartosc = strona.content #zapisujemy jej zawartosc w zmiennej
    link = strona.url #zapisujemy url wylosowanej strony
    soup = BeautifulSoup(zawartosc, 'html.parser') 
    tytul = str(soup.find_all('title')) #z zawartosci wyszukujemy tytulu strony
    print(tytul)
    odp = input("Czy lubisz tytul strony? ").upper() #sprawdzamy czy tytul podoba sie uzytkownikowi
    if odp == 'Y':  #jesli tytul sie podoba wyswietlamy strone
        webbrowser.open(link)
    else: #jesli tytul sie nie podoba funkcja wywoluje sie po raz kolejny
        wikipedia()
wikipedia()

[<title>Paola Igliori - Wikipedia</title>]
